# Using model options in PyBaMM
In this notebook we show how to pass options to models. This allows users to do things such as include extra physics (e.g. thermal effects) or change the macroscopic dimension of the problem (e.g. change from a 1D model to a 2+1D pouch cell model). To see all of the options currently available in PyBaMM, please take a look at the documentation [here](https://pybamm.readthedocs.io/en/latest/source/models/base_models/base_battery_model.html). For more information on combining submodels explicitly to create your own custom model, please see the [Using Submodels notebook](./using-submodels.ipynb).

## Example: Solving the SPMe with a lumped thermal model
PyBaMM is designed to be a flexible modelling package that allows users to easily include different physics within a model without having to start from scratch. In this example, we show how to pass model options to include thermal effects in the SPMe (for more information on the SPMe see [here](./models/SPMe.ipynb)). First we import PyBaMM and any other packages we need

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

Note: you may need to restart the kernel to use updated packages.


We then choose out model options, which a set as a dictionary. We choose to model the behaviour in the particle by assuming the concentration profile is quadratic within the particle. We also choose a lumped thermal model (note that this is fully-coupled, i.e. parameters can depend on temperature). For an in-depth look at the thermal models see the [thermal models notebook](models/thermal-models.ipynb)

In [2]:
options = {"particle": "quadratic profile", "thermal": "lumped"}

We then pass our options to the model 

In [3]:
model = pybamm.lithium_ion.SPMe(options)

We choose to use the parameters from [1]. We then update the heat transfer coefficient and increase the current to approximately 3C (see the [Parameter Values notebook](./parameter-values.ipynb) for more details)

In [4]:
param = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Marquis2019)
param.update(
    {
        "Total heat transfer coefficient [W.m-2.K-1]": 1.0,
        "Current function [A]": 3*0.68
    }
)

We then create and solve a simulation, making sure we pass in our updated parameter values

In [5]:
simulation = pybamm.Simulation(model, parameter_values=param)
simulation.solve([0, 3600])

Finally we plot the terminal voltage and the cell temperature 

In [6]:
simulation.plot([
    "Terminal voltage [V]",
    "X-averaged cell temperature [K]",
    "Cell temperature [K]",
])

interactive(children=(FloatSlider(value=0.0, description='t', max=1163.6363636363637, step=11.636363636363637)…

Note that the variable "X-averaged cell temperature [K]" is the scalar-valued lumped temperature, whereas the variable "Cell temperature [K]" is the value of the lumped temperature broadcasted across the whole cell domain. This type of behaviour is purposefully designed to allow easy comparison of different models and settings. For instance we may wish to compare a simulation that uses a lumped thermal model with a simulation that uses a full thermal model (i.e. one that solves the heat equation in the x-direction). When comparing these two model we could then plot the same variable "Cell temperature [K]" to compare the temperature throughout the cell. 

## References
[1] Marquis, S. G., Sulzer, V.,  Timms, R.,  Please, C. P.,  &  Chapman, S. J. (2019). “An  asymptotic derivation of a single particle model with electrolyte”. [Journal of The Electrochemical Society](https://doi.org/10.1149/2.0341915jes), 166(15), A3693-A3706. 